In [1]:
!wget --no-check-certificate https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip



--2024-02-25 19:33:52--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 173.194.216.207, 173.194.217.207, 108.177.12.207, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|173.194.216.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘cats_and_dogs_filtered.zip’

cats_and_dogs_filte 100%[===================>]  65.43M   194MB/s    in 0.3s    

2024-02-25 19:33:52 (194 MB/s) - ‘cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [3]:
import zipfile

local_zip = './cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall()

zip_ref.close()


In [6]:
import os

base_dir = 'cats_and_dogs_filtered'

print("Contents of base directory:")
print(os.listdir(base_dir))

print("\nContents of train directory:")
print(os.listdir(f'{base_dir}/train'))

print("\nContents of validation directory:")
print(os.listdir(f'{base_dir}/validation'))

Contents of base directory:
['validation', 'vectorize.py', 'train']

Contents of train directory:
['dogs', 'cats']

Contents of validation directory:
['dogs', 'cats']


In [8]:
import os

train_dir = os.path.join(base_dir,'train')
validation_dir= os.path.join(base_dir,'validation')

train_cats_dir = os.path.join(train_dir,'cats')
train_dogs_dir = os.path.join(train_dir, 'dogs')

validation_cats_dir = os.path.join(validation_dir,'cats')
validation_dogs_dir = os.path.join(validation_dir, 'dogs')


In [9]:
train_cat_fnames= os.listdir( train_cats_dir)
train_dog_fnames= os.listdir( train_dogs_dir)

print(train_cat_fnames[:10])
print(train_dog_fnames[:10])

['cat.594.jpg', 'cat.749.jpg', 'cat.734.jpg', 'cat.497.jpg', 'cat.376.jpg', 'cat.662.jpg', 'cat.246.jpg', 'cat.604.jpg', 'cat.463.jpg', 'cat.983.jpg']
['dog.985.jpg', 'dog.74.jpg', 'dog.478.jpg', 'dog.180.jpg', 'dog.584.jpg', 'dog.880.jpg', 'dog.748.jpg', 'dog.840.jpg', 'dog.873.jpg', 'dog.507.jpg']


In [10]:
print("total training cat images:",len(os.listdir( train_cats_dir)))
print("total training dogs images:", len(os.listdir( train_dogs_dir)))

print("total validation cats images", len(os.listdir( validation_cats_dir)))
print("total validation dogs images", len(os.listdir(validation_dogs_dir)))

total training cat images: 1000
total training dogs images: 1000
total validation cats images 500
total validation dogs images 500


In [11]:
%matplotlib inline
import matplotlib.image as mping
import matplotlib.pyplot as plt

nrows = 4
ncols = 4

pic_index = 0


In [14]:
fig = plt.gcf()
fig.set_size_inches(ncols*4, nrows*4)
pic_index=0

next_cat_pix =[os.path.join(train_cats_dir,fname)
               for fname in train_cat_fnames[ pic_index-8:pic_index]]

next_dog_pix = [os.path.join(train_dogs_dir,fname)
                 for fname in train_dog_fnames[ pic_index-8:pic_index]]

for i, image_path in enumerate(next_cat_pix+next_dog_pix):
  sp=plt.subplot(nrows,ncols,i+1)
  sp.axix('off')

  img=mping.imread(img_path)
  plt.imshow(img)

plt.show()


<Figure size 1600x1600 with 0 Axes>

In [18]:
import tensorflow as tf

model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(16,(3,3), activation= 'relu', input_shape = (150,150,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512,activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 148, 148, 16)      448       
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 74, 74, 16)        0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 72, 72, 32)        4640      
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 36, 36, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_4 (Conv2D)           (None, 34, 34, 64)        18496     
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 17, 17, 64)        0

In [20]:
from tensorflow.keras.optimizers import RMSprop

model.compile(optimizer=RMSprop(learning_rate=0.001),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [23]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator( rescale = 1.0/255.)
test_datagen = ImageDataGenerator( rescale  = 1.0/255.)

train_generator = train_datagen.flow_from_directory(train_dir,batch_size=20,class_mode='binary',
                                                    target_size=(150,150))

validation_generator = test_datagen.flow_from_directory(validation_dir,batch_size=20,class_mode='binary',
                                                        target_size=(150,150))

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [24]:
history =model.fit(
    train_generator,
    epochs=15,
    validation_data= validation_generator,
    verbose=2
)

Epoch 1/15
100/100 - 57s - loss: 0.7242 - accuracy: 0.5465 - val_loss: 0.6704 - val_accuracy: 0.5620 - 57s/epoch - 570ms/step
Epoch 2/15
100/100 - 60s - loss: 0.6630 - accuracy: 0.6155 - val_loss: 0.6150 - val_accuracy: 0.6620 - 60s/epoch - 599ms/step
Epoch 3/15
100/100 - 54s - loss: 0.5813 - accuracy: 0.7005 - val_loss: 0.6093 - val_accuracy: 0.6740 - 54s/epoch - 545ms/step
Epoch 4/15
100/100 - 59s - loss: 0.5298 - accuracy: 0.7330 - val_loss: 0.5847 - val_accuracy: 0.6710 - 59s/epoch - 590ms/step
Epoch 5/15
100/100 - 58s - loss: 0.4803 - accuracy: 0.7610 - val_loss: 0.6127 - val_accuracy: 0.6530 - 58s/epoch - 583ms/step
Epoch 6/15
100/100 - 57s - loss: 0.4304 - accuracy: 0.7890 - val_loss: 0.5420 - val_accuracy: 0.7320 - 57s/epoch - 567ms/step
Epoch 7/15
100/100 - 59s - loss: 0.3435 - accuracy: 0.8525 - val_loss: 0.6112 - val_accuracy: 0.7110 - 59s/epoch - 585ms/step
Epoch 8/15
100/100 - 60s - loss: 0.2770 - accuracy: 0.8815 - val_loss: 0.5897 - val_accuracy: 0.7170 - 60s/epoch - 601

In [28]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array
uploaded =files.upload()
for fn in uploaded.keys():
  path = '/content/'+fn
  img =load_img(path,target_size =(150,150))
  x= img_to_array(img)
  x/= 255
  x= np.expand_dims(x,axis=0)
  images=np.vstack([x])
  classes = model.predict(images,batch_size=10)
  print(classes[0])

  if classes[0]>0.5:
    print(fn+"is a dog ")
  else :
    print(fn+ "is a cat")

Saving 16.jpg to 16.jpg
1/1 [==============================] - 0s 39ms/step
[0.99991834]
16.jpgis a dog 
